# Automatizando o Pré-processamento com SKLearn

## Etapa de análise exploratória
Não repetiremos aqui a análise exploratória feita anteriormente. Faremos apenas a carga dos dados e o pré-processamento.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('day.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


## Etapa de pré-processamento

### Separando os conjuntos de treino e teste

In [4]:
from sklearn.model_selection import train_test_split

# separando os conjuntos de dados de treino e teste
df_treino, df_teste = train_test_split(df, test_size=0.2, random_state=42)

# separando a coluna alvo do conjunto de treino
df_treino_labels = df_treino['cnt'].copy()
df_treino        = df_treino.drop(columns='cnt')

# separando a coluna alvo do conjunto de teste
df_teste_labels = df_teste['cnt'].copy()
df_teste        = df_teste.drop(columns='cnt')

### Automação do processo de pré-processamento

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [6]:
nomes_atributos_numericos   = ['temp', 'hum', 'windspeed']
nomes_atributos_categoricos = ['season','mnth','weekday','weathersit']
nomes_atributos_binarios    = ['holiday','workingday']

In [7]:
pipeline_atr_numericos = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

preproc_completo = ColumnTransformer([
    ('numericos',   pipeline_atr_numericos, nomes_atributos_numericos),
    ('binarios',    'passthrough',          nomes_atributos_binarios),
    ('categoricos', OneHotEncoder(),        nomes_atributos_categoricos),
    ], 
    sparse_threshold=0)

In [8]:
# pre-processamento do conjunto de treino
X_treino = preproc_completo.fit_transform(df_treino)

# chamar fit_transform é o mesmo que chamar 
# primeiro fit para aprender os parâmetros
# e depois chamar transform para realizar 
# as transformações com base nos parâmetros
# previamente aprendidos, ou seja, chamar:

# preproc_completo.fit(df_treino)
# X_treino = preproc_completo.transform(df_treino)

In [9]:
X_treino[0:2]

array([[-0.86461022,  0.22342426,  1.9570449 ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.73728848,  2.17101774,  0.0267188 ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ]])

In [10]:
# pre-processamento do conjunto de teste
X_teste = preproc_completo.transform(df_teste)

In [11]:
X_teste[0:2]

array([[-0.13416911,  0.72208642, -0.21401277,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-1.72667406, -1.36120952,  1.12576527,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ]])

In [12]:
y_treino = df_treino_labels.values.reshape(-1,1)
y_teste  = df_teste_labels.values.reshape(-1,1)